In [5]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import csv

timesteps = 3

num_classes = timesteps  # follow timestep
data_dim = 15
batchsize = 32 # number of data in a batch
drop = 0.2

#%%
# load dataset
dataset = pd.read_csv("/content/Index Time Series Multi Variate (Edited).csv")

df = pd.DataFrame(dataset)

data_size = len(dataset) # datasize selected must have both attack and normal data.
data_resize = int(data_size//timesteps) #data_size/timesteps using // because round down, example 10/3=3
data_trunc_size = data_resize * timesteps # remove extra rows for so that data can be divided by timesteps

df = df[:data_trunc_size]
df.head()

,Date,Close_10year_treasury,Volume_10year_treasury,Close_copper,Volume_copper,Close_gold,Volume_gold,Close_hk_index,Volume_hk_index,Close_oil,Volume_oil,Close_s&p,Volume_s&p,Value_us_sgd,Increase
0,3/27/2015,1.948,0,2.7750,483,1199.800049,68,24486.19922,1610426400,48.869999,515277,2061.020020,3008550000,1.3690,True
1,3/30/2015,1.963,0,2.7875,714,1184.800049,172260,24855.11914,2574119600,48.680000,384092,2086.239990,2917690000,1.3759,True
2,3/31/2015,1.934,0,2.7470,671,1183.099976,37990,24900.89063,2557002200,47.599998,355866,2067.889893,3376550000,1.3716,False
3,4/1/2015,1.868,0,2.7565,401,1208.099976,3409,25082.75000,1713463600,50.090000,365748,2059.689941,3543270000,1.3622,True
4,4/2/2015,1.904,0,2.7480,550,1200.900024,1632,25275.64063,2106782000,49.139999,451104,2066.959961,3095960000,1.3567,True


In [6]:
df.shape

(1158, 15)

In [7]:
df.head().T

,0,1,2,3,4
Date,3/27/2015,3/30/2015,3/31/2015,4/1/2015,4/2/2015
Close_10year_treasury,1.948,1.963,1.934,1.868,1.904
Volume_10year_treasury,0,0,0,0,0
Close_copper,2.775,2.7875,2.747,2.7565,2.748
Volume_copper,483,714,671,401,550
Close_gold,1199.8,1184.8,1183.1,1208.1,1200.9
Volume_gold,68,172260,37990,3409,1632
Close_hk_index,24486.2,24855.1,24900.9,25082.8,25275.6
Volume_hk_index,1610426400,2574119600,2557002200,1713463600,2106782000
Close_oil,48.87,48.68,47.6,50.09,49.14


In [8]:

# Remove the dates, therefore X start from 1
X = df.iloc[:,1:-1].values
Y = df.iloc[:,-1].values

print(X[:5])
print(Y[:5])

[[1.94800000e+00 0.00000000e+00 2.77500000e+00 4.83000000e+02
  1.19980005e+03 6.80000000e+01 2.44861992e+04 1.61042640e+09
  4.88699990e+01 5.15277000e+05 2.06102002e+03 3.00855000e+09
  1.36900000e+00]
 [1.96300000e+00 0.00000000e+00 2.78750000e+00 7.14000000e+02
  1.18480005e+03 1.72260000e+05 2.48551191e+04 2.57411960e+09
  4.86800000e+01 3.84092000e+05 2.08623999e+03 2.91769000e+09
  1.37590000e+00]
 [1.93400000e+00 0.00000000e+00 2.74700000e+00 6.71000000e+02
  1.18309998e+03 3.79900000e+04 2.49008906e+04 2.55700220e+09
  4.75999980e+01 3.55866000e+05 2.06788989e+03 3.37655000e+09
  1.37160000e+00]
 [1.86800000e+00 0.00000000e+00 2.75650000e+00 4.01000000e+02
  1.20809998e+03 3.40900000e+03 2.50827500e+04 1.71346360e+09
  5.00900000e+01 3.65748000e+05 2.05968994e+03 3.54327000e+09
  1.36220000e+00]
 [1.90400000e+00 0.00000000e+00 2.74800000e+00 5.50000000e+02
  1.20090002e+03 1.63200000e+03 2.52756406e+04 2.10678200e+09
  4.91399990e+01 4.51104000e+05 2.06695996e+03 3.09596000e+0

In [9]:
# Reduce the size of data so that the data can be divided by time step
# split into input (X) and output (Y) variables


# required format for lstm
X_shaped = X.reshape(data_resize, timesteps, data_dim-2).astype('float32')
Y_shaped = Y.reshape(data_resize,timesteps).astype('bool')
print("X shape is : {}".format(X_shaped.shape))
print("Y shape is : {}".format(Y_shaped.shape))

X shape is : (386, 3, 13)
Y shape is : (386, 3)


In [10]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_shaped, Y_shaped, test_size=0.3)

# LSTM

In [12]:


# Create the model
# expected input data shape: (batch_size, timesteps, data_dim)
# Dropout used to prevent over-fitting.
# Input shape will infer the batch size by itself
# We are using binary cross entropy even when num_class can be > 1 because this is a binary classification on an array
# For multiclass classification, use categorical cross-entropy
model = Sequential()
model.add(LSTM(36, return_sequences=True, input_shape=(timesteps, data_dim-2)))  # returns a sequence of vectors of dimension 40
model.add(Dropout(drop))
model.add(LSTM(36,return_sequences=True))  # returns a sequence of vectors of dimension 40
model.add(Dropout(drop))
model.add(LSTM(36,return_sequences=True))  # returns a sequence of vectors of dimension 40
model.add(Dropout(drop))
model.add(LSTM(36))  # return a single vector of dimension 40
model.add(Dropout(drop))
model.add(Dense(num_classes, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# Train the model
model.fit(X_train, y_train, batch_size= batchsize, epochs=100, validation_data= (X_test, y_test))

# Evaluate the model
# Returns you the accuracy and loss
loss, acc = model.evaluate(X_train, y_train,timesteps)
print("Keras: \n%s: %.2f%%" % (model.metrics_names[1], acc*100))

# Shape of prediction is nrow * timestep
# Result would be that same as keras evaluate
proba = model.predict(X_test)

# proba is the probability. Here we set threshold as 0.5 to be considered true.
print('predictions shape:', proba.shape)
y_pred = proba > 0.5
# Reshape to a single dimension for comparison and to create confusion matrix
y_pred_single_dim = y_pred.reshape(proba.shape[0]*proba.shape[1])
y_test_single_dim = y_test.reshape(y_test.shape[0]*y_test.shape[1])
print(y_pred_single_dim)
print(y_test_single_dim)
matrix = confusion_matrix(y_test_single_dim, y_pred_single_dim)
print("Keras: \n%s: %.2f%%" % (model.metrics_names[1], sum(y_pred_single_dim==y_test_single_dim)/len(y_test_single_dim)*100))
print(matrix)


Epoch 1/100
9/9 [==============================] - 1s 144ms/step - loss: 0.6937 - accuracy: 0.4556 - val_loss: 0.6921 - val_accuracy: 0.6724
Epoch 2/100
9/9 [==============================] - 0s 9ms/step - loss: 0.6930 - accuracy: 0.6444 - val_loss: 0.6919 - val_accuracy: 0.6724
Epoch 3/100
9/9 [==============================] - 0s 9ms/step - loss: 0.6929 - accuracy: 0.6444 - val_loss: 0.6920 - val_accuracy: 0.6724
Epoch 4/100
9/9 [==============================] - 0s 10ms/step - loss: 0.6930 - accuracy: 0.6444 - val_loss: 0.6919 - val_accuracy: 0.6724
Epoch 5/100
9/9 [==============================] - 0s 10ms/step - loss: 0.6932 - accuracy: 0.6444 - val_loss: 0.6917 - val_accuracy: 0.6724
Epoch 6/100
9/9 [==============================] - 0s 9ms/step - loss: 0.6926 - accuracy: 0.6444 - val_loss: 0.6913 - val_accuracy: 0.6724
Epoch 7/100
9/9 [==============================] - 0s 10ms/step - loss: 0.6928 - accuracy: 0.6444 - val_loss: 0.6911 - val_accuracy: 0.6724
Epoch 8/100
9/9 [=====

# RNN

In [13]:
from tensorflow.keras.layers import SimpleRNN

# Create the model
# expected input data shape: (batch_size, timesteps, data_dim)
# Dropout used to prevent over-fitting.
# Input shape will infer the batch size by itself
# We are using binary cross entropy even when num_class can be > 1 because this is a binary classification on an array
# For multiclass classification, use categorical cross-entropy
model = Sequential()
model.add(SimpleRNN(36, return_sequences=True, input_shape=(timesteps, data_dim-2)))  # returns a sequence of vectors of dimension 40
model.add(Dropout(drop))
model.add(SimpleRNN(36,return_sequences=True))  # returns a sequence of vectors of dimension 40
model.add(Dropout(drop))
model.add(SimpleRNN(36,return_sequences=True))  # returns a sequence of vectors of dimension 40
model.add(Dropout(drop))
model.add(SimpleRNN(36))  # return a single vector of dimension 40
model.add(Dropout(drop))
model.add(Dense(num_classes, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# Train the model
model.fit(X_train, y_train, batch_size= batchsize, epochs=1, validation_data= (X_test, y_test))

# Evaluate the model
# Returns you the accuracy and loss
loss, acc = model.evaluate(X_train, y_train,timesteps)
print("Keras: \n%s: %.2f%%" % (model.metrics_names[1], acc*100))

# Shape of prediction is nrow * timestep
# Result would be that same as keras evaluate
proba = model.predict(X_test)

# proba is the probability. Here we set threshold as 0.5 to be considered true.
print('predictions shape:', proba.shape)
y_pred = proba > 0.5
# Reshape to a single dimension for comparison and to create confusion matrix
y_pred_single_dim = y_pred.reshape(proba.shape[0]*proba.shape[1])
y_test_single_dim = y_test.reshape(y_test.shape[0]*y_test.shape[1])
print(y_pred_single_dim)
print(y_test_single_dim)
matrix = confusion_matrix(y_test_single_dim, y_pred_single_dim)
print("Keras: \n%s: %.2f%%" % (model.metrics_names[1], sum(y_pred_single_dim==y_test_single_dim)/len(y_test_single_dim)*100))
print(matrix)


90/90 [==============================] - 0s 2ms/step - loss: 0.7059 - accuracy: 0.1889
Keras: 
accuracy: 18.89%
predictions shape: (116, 3)
[False False False False False False False False False False False  True
 False False  True False False False False False  True False False  True
 False False False False False  True  True False False False False  True
 False False False False False  True False False  True False False  True
 False False False False False False False False False False False  True
 False False  True False False  True False False False False False  True
 False False False False False False False False  True False False False
 False False False False False  True False False  True False False  True
 False False  True False False False False False  True  True False False
 False False  True False False False False False  True False False False
 False False False False False False False False  True False False  True
 False False  True  True False False False False  True Fa

# GRU (simpler LSTM)

In [14]:
from tensorflow.keras.layers import GRU

# Create the model
# expected input data shape: (batch_size, timesteps, data_dim)
# Dropout used to prevent over-fitting.
# Input shape will infer the batch size by itself
# We are using binary cross entropy even when num_class can be > 1 because this is a binary classification on an array
# For multiclass classification, use categorical cross-entropy
model = Sequential()
model.add(GRU(36, return_sequences=True, input_shape=(timesteps, data_dim-2)))  # returns a sequence of vectors of dimension 40
model.add(Dropout(drop))
model.add(GRU(36,return_sequences=True))  # returns a sequence of vectors of dimension 40
model.add(Dropout(drop))
model.add(GRU(36,return_sequences=True))  # returns a sequence of vectors of dimension 40
model.add(Dropout(drop))
model.add(GRU(36))  # return a single vector of dimension 40
model.add(Dropout(drop))
model.add(Dense(num_classes, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# Train the model
model.fit(X_train, y_train, batch_size= batchsize, epochs=1, validation_data= (X_test, y_test))

# Evaluate the model
# Returns you the accuracy and loss
loss, acc = model.evaluate(X_train, y_train,timesteps)
print("Keras: \n%s: %.2f%%" % (model.metrics_names[1], acc*100))

# Shape of prediction is nrow * timestep
# Result would be that same as keras evaluate
proba = model.predict(X_test)

# proba is the probability. Here we set threshold as 0.5 to be considered true.
print('predictions shape:', proba.shape)
y_pred = proba > 0.5
# Reshape to a single dimension for comparison and to create confusion matrix
y_pred_single_dim = y_pred.reshape(proba.shape[0]*proba.shape[1])
y_test_single_dim = y_test.reshape(y_test.shape[0]*y_test.shape[1])
print(y_pred_single_dim)
print(y_test_single_dim)
matrix = confusion_matrix(y_test_single_dim, y_pred_single_dim)
print("Keras: \n%s: %.2f%%" % (model.metrics_names[1], sum(y_pred_single_dim==y_test_single_dim)/len(y_test_single_dim)*100))
print(matrix)

90/90 [==============================] - 0s 2ms/step - loss: 0.6927 - accuracy: 0.6444
Keras: 
accuracy: 64.44%
predictions shape: (116, 3)
[ True False False  True False False  True False False  True False False
  True False False  True False False  True False False  True False False
  True False False  True False False  True False False  True False False
  True False False  True False False  True False False  True False False
  True False False  True False False  True False False  True False False
  True False False  True False False  True False False  True False False
  True False False  True False False  True False False  True False False
  True False False  True False False  True False False  True False False
  True False False  True False False  True False False  True False False
  True False False  True False False  True False False  True False False
  True False False  True False False  True False False  True False False
  True False False  True False False  True False False  T